# 🔫 Evaluate a zero-shot NER with Flair

In this tutorial you will learn how to analyze and validate NER predictions from the new zero-shot model provided by the Flair NLP library with Argilla.

- 🛠 Useful for quickly bootstrapping a training set (using Argilla [*Annotation Mode*](../../reference/webapp/features.html#annotate)) as well as integrating with weak-supervision workflows.
- 👁 We will use a challenging, exciting dataset: wnut_17 (more info below).
- 🔮 You will be able to see and work with the obtained predictions.

![labelling-tokenclassification-flair-fewshot](../../_static/tutorials/labelling-tokenclassification-flair-fewshot/labelling-tokenclassification-flair-fewshot.png)

## Introduction

This tutorial will show you how to work with Named Entity Recognition (NER), Flair and Argilla. But, what is NER?

According to [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2021/06/part-10-step-by-step-guide-to-master-nlp-named-entity-recognition/), 
"NER is a **natural language processing technique** that can automatically scan entire articles and pull out some fundamental entities in a text and classify them into predefined categories". These entities can be names, quantities, dates and times, amounts of money/currencies, and much more. 

On the other hand, [Flair](https://github.com/flairNLP/flair) is a library which facilitates the application of NLP models to NER and other NLP techniques in many different languages. It is not only a powerful library, but also intuitive. 

Thanks to these resources and the [*Annotation Mode*](../../reference/webapp/features.html#annotate) of *Argilla*, we can quickly build up a data set to train a domain-specific model.

## Running Argilla

For this tutorial, you will need to have an Argilla server running. There are two main options for deploying and running Argilla:

1. [Deploy Argilla on Hugging Face Spaces](../../getting_started/installation/deployments/huggingface-spaces.html): This is the fastest option and the recommended choice for connecting to external notebooks (e.g., Google Colab) if you have an account on Hugging Face.

2. [Launch Argilla using Argilla's quickstart Docker image](../../getting_started/quickstart.ipynb): This is the recommended option if you want Argilla running on your local machine. Note that this option will only let you run the tutorial locally and not with an external notebook service.

For more information on deployment options, please check the Deployment section of the documentation.

<div class="alert alert-info">

Tip
    
This tutorial is a Jupyter Notebook. There are two options to run it:

- Use the Open in Colab button at the top of this page. This option allows you to run the notebook directly on Google Colab.
- Download the .ipynb file by clicking on the View source link at the top of the page. This option allows you to download the notebook and run it on your local machine or on a Jupyter notebook tool of your choice.
</div>

## Setup

For this tutorial, you'll need to install the Argilla client and a few third party libraries using `pip`:

In [ ]:
%pip install "argilla" "datasets~=2.6.0" "flair~=0.11.0" -qqq

Let's import the Argilla module for reading and writing data:

In [ ]:
import argilla as rg

If you are running Argilla using the Docker quickstart image or Hugging Face Spaces, you need to init the Argilla client with the URL and API_KEY:

In [ ]:
# Replace api_url with the url to your HF Spaces URL if using Spaces
# Replace api_key if you configured a custom API key
rg.init(
    api_url="http://localhost:6900", 
    api_key="team.apikey"
)

Finally, let's include the imports we need:

In [ ]:
from datasets import load_dataset
from flair.models import TARSTagger
from flair.data import Sentence

## 1. Load the `wnut_17` dataset

In this example, we'll use a challenging NER dataset, the **"WNUT 17: Emerging and Rare entity recognition"** , which focuses on unusual, previously-unseen entities in the context of emerging discussions. This dataset is useful for getting a sense of the quality of our zero-shot predictions.

Let's load the test set from the [Hugging Face Hub](https://huggingface.co/datasets/wnut_17):

In [ ]:
# download data set
dataset = load_dataset("wnut_17", split="test")

# define labels
labels = ["corporation", "creative-work", "group", "location", "person", "product"]

## 2. Configure Flair TARSTagger

Now let's configure our NER model, following [Flair's  documentation](https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_10_TRAINING_ZERO_SHOT_MODEL.md#use-case-2-zero-shot-named-entity-recognition-ner-with-tars):

In [ ]:
# load zero-shot NER tagger
tars = TARSTagger.load("tars-ner")

# define labels for named entities using wnut labels
tars.add_and_switch_to_new_task("task 1", labels, label_type="ner")

Let's test it with one example!

In [ ]:

# wrap our tokens in a flair Sentence
sentence = Sentence(" ".join(dataset[0]["tokens"]))

# add predictions to our sentence
tars.predict(sentence)

# extract predicted entities into a list of tuples (entity, start_char, end_char)
[
    (entity.get_labels()[0].value, entity.start_position, entity.end_position)
    for entity in sentence.get_spans("ner")
]

## 3. Predict over `wnut_17` and log into `argilla`

Now, let's log the predictions in Argilla:

In [ ]:
# build records for the first 100 examples
records = []

for record in dataset.select(range(100)):
    input_text = " ".join(record["tokens"])

    sentence = Sentence(input_text)
    tars.predict(sentence)
    prediction = [
        (entity.get_labels()[0].value, entity.start_position, entity.end_position)
        for entity in sentence.get_spans("ner")
    ]

    # building TokenClassificationRecord
    records.append(
        rg.TokenClassificationRecord(
            text=input_text,
            tokens=[token.text for token in sentence],
            prediction=prediction,
            prediction_agent="tars-ner",
        )
    )

# log the records to Argilla
rg.log(records, name="tars_ner_wnut_17", metadata={"split": "test"})

Now you can see the results obtained! With the annotation mode, you can change, add, validate or discard your results. Statistics are also available, to better monitor your records!

## Summary

Getting predictions with a zero-shot approach can be very helpful to guide humans in their annotation process.
Especially for NER tasks, Argilla makes it very easy to explore and correct those predictions thanks to its **Annotation Mode** 😎.